In [1]:
import pandas as pd
import numpy as np
import os

from pandas import DataFrame
from matplotlib import colors as mcolors
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

os.chdir("/home/william/Desktop")

ImportError: Unable to import required dependencies:
pytz: No module named 'pytz'

In [ ]:
datosTicTacToe = pd.read_csv("tic-tac-toe.csv")

In [ ]:
for x in datosTicTacToe:
    if x != " Gana-x":
        datosTicTacToe[x] = datosTicTacToe[x].astype('category')
datosTicTacToe.dtypes

datosTicTacToe = datosTicTacToe.replace({'x': 0, 'o': 1, 'b': 2})


In [ ]:
class PrediccionBase:
    def __init__(self, datos):
        self.__datos = self.cleaning(datos)
        self.__precisionGlobal = 0
        self.__error_global = 0
        self.__verdaderosNegativos = 0
        self.__falsosPositivos = 0
        self.__falsosNegativos = 0
        self.__verdaderosPositivos = 0
        self.__reporte = 0
        self.__precision_category = 0

    @property
    def datos(self):
        return self.__datos

    @property
    def reporte(self):
        return self.__reporte

    def cleaning(self, datos):
        datos = datos.replace({'x': 0, 'o': 1, 'b': 2})
        return datos

    def entrenamiento(self):
        pass

    def generacionReporte(self, nombreDelModelo):
        dict = {
            "Modelo": [nombreDelModelo],
            "Precision Global": [self.__precisionGlobal],
            "Error Global": [self.__error_global],
            "Verdaderos Positivos": [self.__verdaderosPositivos],
            "Verdaderos Negativos": [self.__verdaderosNegativos],
            "Falsos Negativos": [self.__falsosNegativos],
            "Falsos Positivos": [self.__falsosPositivos]}
        self.__reporte = pd.DataFrame(dict).join(self.__precision_category)

    def analsis(self, MC, modelo):
        self.__verdaderosNegativos, self.__falsosPositivos, self.__falsosNegativos, self.__verdaderosPositivos = MC.ravel()
        self.__precisionGlobal = np.sum(MC.diagonal()) / np.sum(MC)
        self.__error_global = 1 - self.__precisionGlobal
        self.__precision_category = pd.DataFrame(MC.diagonal() / np.sum(MC, axis=1)).T
        self.__precision_category.columns = ["Precision Positiva (PP)", "Precision Negativa (PN)"]
        self.generacionReporte(modelo)
        return {"Matriz de Confusión": MC,
                "Precisión Global": self.__precisionGlobal,
                "Error Global": self.__error_global,
                "Precisión por categoría": self.__precision_category}


class PrediccionKNeighbors(PrediccionBase):
    def __init__(self, datos):
        super().__init__(datos)
        self.__instancia_potenciacion = None
        self.__x_test = None

    @property
    def instancia(self):
        return self.__instancia_potenciacion

    @property
    def x_test(self):
        return self.__x_test

    def entrenamiento(self, nucleo="auto", n_neighbors=3, train_size=0.80, entrenamiento="KNeighbors"):
        x = self.datos.iloc[:, :-1]
        y = self.datos.iloc[:, -1]
        X_train, self.__x_test, y_train, y_test = train_test_split(x, y, train_size=train_size, random_state=0)
        self.__instancia_potenciacion = KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=nucleo)
        self.__instancia_potenciacion.fit(X_train, y_train)
        prediccion = self.__instancia_potenciacion.predict(self.__x_test)
        MC = confusion_matrix(y_test, prediccion)
        indices = self.analsis(MC, entrenamiento)
        # for k in indices:
        #     print("\n%s:\n%s" % (k, str(indices[k])))


class PrediccionADABoosting(PrediccionBase):
    def __init__(self, datos):
        super().__init__(datos)
        self.__instancia = None
        self.__instancia_potenciacion = None
        self.__x_train = []

    @property
    def instancia(self):
        return self.__instancia_potenciacion

    @property
    def x_test(self):
        return self.__x_test

    def obtenerVariablesImportantes(self):
        importancia = self.__instancia_potenciacion.feature_importances_
        print(importancia)
        etiquetas = self.__x_train.columns.values
        y_pos = np.arange(len(etiquetas))
        plt.figure(figsize=(10, 8))
        plt.barh(y_pos, importancia, align='center', alpha=0.5)
        plt.yticks(y_pos, etiquetas)

    def entrenamiento(self, train_size=0.75, criterion="gini", splitter="best", min_samples_split=2,
                      entretenamiento="Bosques Aleatorios ADA Boosting"):
        x = self.datos.iloc[:, :-1]
        y = self.datos.iloc[:, -1]
        self.__instancia = DecisionTreeClassifier(min_samples_split=min_samples_split, max_depth=None,
                                                  criterion=criterion, splitter=splitter)
        self.__instancia_potenciacion = AdaBoostClassifier(base_estimator=self.__instancia,
                                                           n_estimators=100, random_state=0)
        self.__x_train, self.__x_test, y_train, y_test = train_test_split(x, y, train_size=train_size, random_state=0)
        self.__instancia_potenciacion.fit(self.__x_train, y_train)
        prediccion = self.__instancia_potenciacion.predict(self.__x_test)
        MC = confusion_matrix(y_test, prediccion)
        indices = self.analsis(MC, entretenamiento)
        # for k in indices:
        #     print("\n%s:\n%s" % (k, str(indices[k])))


class PrediccionXGBoosting(PrediccionBase):
    def __init__(self, datos):
        super().__init__(datos)
        self.__instancia = None
        self.__instancia_potenciacion = None
        self.__x_train = []

    @property
    def x_test(self):
        return self.__x_test
    @property
    def instancia(self):
        return self.__instancia_potenciacion

    def obtenerVariablesImportantes(self):
        importancia = self.__instancia_potenciacion.feature_importances_
        print(importancia)
        etiquetas = self.__x_train.columns.values
        y_pos = np.arange(len(etiquetas))
        plt.figure(figsize=(10, 8))
        plt.barh(y_pos, importancia, align='center', alpha=0.5)
        plt.yticks(y_pos, etiquetas)

    def entrenamiento(self, train_size=0.75, n_estimators=10, random_state=0, min_samples_split=2,
                      entrenamiento="Bosques Aleatorios XG Boosting"):
        x = self.datos.iloc[:, :-1]
        y = self.datos.iloc[:, -1]

        self.__instancia_potenciacion = GradientBoostingClassifier(n_estimators=n_estimators, random_state=n_estimators,
                                                                   min_samples_split=min_samples_split, max_depth=None)

        self.__x_train, self.__x_test, y_train, y_test = train_test_split(x, y, train_size=train_size, random_state=0)
        self.__instancia_potenciacion.fit(self.__x_train, y_train)
        prediccion = self.__instancia_potenciacion.predict(self.__x_test)
        MC = confusion_matrix(y_test, prediccion)
        indices = self.analsis(MC, entrenamiento)
        # for k in indices:
        #     print("\n%s:\n%s" % (k, str(indices[k])))


class PrediccionRandomForest(PrediccionBase):
    def __init__(self, datos):
        super().__init__(datos)
        self.__instancia = None
        self.__instancia_potenciacion = None
        self.__x_train = []

    @property
    def instancia(self):
        return self.__instancia_potenciacion

    @property
    def x_test(self):
        return self.__x_test

    def obtenerVariablesImportantes(self):
        importancia = self.__instancia_potenciacion.feature_importances_
        print(importancia)
        etiquetas = self.__x_train.columns.values
        y_pos = np.arange(len(etiquetas))
        plt.figure(figsize=(10, 8))
        plt.barh(y_pos, importancia, align='center', alpha=0.5)
        plt.yticks(y_pos, etiquetas)

    def entrenamiento(self, train_size=0.75, n_estimators=10, entrenamiento="Bosques Aleatorios"):
        x = self.datos.iloc[:, :-1]
        y = self.datos.iloc[:, -1]

        self.__instancia_potenciacion = RandomForestClassifier(n_estimators=n_estimators, random_state=0)

        self.__x_train, self.__x_test, y_train, y_test = train_test_split(x, y, train_size=train_size, random_state=0)
        self.__instancia_potenciacion.fit(self.__x_train, y_train)
        prediccion = self.__instancia_potenciacion.predict(self.__x_test)
        MC = confusion_matrix(y_test, prediccion)
        indices = self.analsis(MC, entrenamiento)
        # for k in indices:
        #     print("\n%s:\n%s" % (k, str(indices[k])))

In [ ]:
import seaborn as sns
class Analisis_Predictivo:

    def __init__(self,datos:DataFrame, predecir:str, predictoras = [],
                 modelo = None,train_size = 80,random_state = None):
        '''
        datos: Datos completos y listos para construir un modelo

        modelo: Instancia de una Clase de un método de clasificación(KNN,Árboles,SVM,etc).
        Si no especifica un modelo no podrá utilizar el método fit_n_review()

        predecir: Nombre de la variable a predecir

        predictoras: Lista de los nombres de las variables predictoras.
        Si vacío entonces utiliza todas las variables presentes excepto la variable a predecir.

        train_size: Proporción de la tabla de entrenamiento respecto a la original.

        random_state: Semilla aleatoria para la división de datos(training-testing).
        '''
        self.datos = datos
        self.predecir = predecir
        self.predictoras = predictoras
        self.modelo = modelo
        self.random_state = random_state
        if modelo != None:
            self.train_size = train_size
            self._training_testing()


    def _training_testing(self):
        if len(self.predictoras) == 0:
            X = self.datos.drop(columns=[self.predecir])
        else:
            X = self.datos[self.predictoras]

        y = self.datos[self.predecir].values

        train_test = train_test_split(X, y, train_size=self.train_size,
                                      random_state=self.random_state)
        self.X_train, self.X_test,self.y_train, self.y_test = train_test


    def fit_predict(self):
        if self.modelo != None:
            self.modelo.fit(self.X_train,self.y_train)
            return self.modelo.predict(self.X_test)

    def fit_predict_resultados(self, imprimir = True):
        if(self.modelo != None):
            y = self.datos[self.predecir].values
            prediccion = self.fit_predict()
            MC = confusion_matrix(self.y_test, prediccion)
            indices = self.indices_general(MC,list(np.unique(y)))
            if imprimir == True:
                for k in indices:
                    print("\n%s:\n%s"%(k,str(indices[k])))

            return indices

    def indices_general(self,MC, nombres = None):
        "Método para calcular los índices de calidad de la predicción"
        precision_global = np.sum(MC.diagonal()) / np.sum(MC)
        error_global = 1 - precision_global
        precision_categoria  = pd.DataFrame(MC.diagonal()/np.sum(MC,axis = 1)).T
        if nombres!=None:
            precision_categoria.columns = nombres
        return {"Matriz de Confusión":MC,
                "Precisión Global":precision_global,
                "Error Global":error_global,
                "Precisión por categoría":precision_categoria}

    def distribucion_variable_predecir(self):
        "Método para graficar la distribución de la variable a predecir"
        variable_predict = self.predecir
        data = self.datos
        colors = list(dict(**mcolors.CSS4_COLORS))
        df = pd.crosstab(index=data[variable_predict],columns="valor") / data[variable_predict].count()
        fig = plt.figure(figsize=(10,9))
        g = fig.add_subplot(111)
        countv = 0
        titulo = "Distribución de la variable %s" % variable_predict
        for i in range(df.shape[0]):
            g.barh(1,df.iloc[i],left = countv, align='center',color=colors[11+i],label= df.iloc[i].name)
            countv = countv + df.iloc[i]
        vals = g.get_xticks()
        g.set_xlim(0,1)
        g.set_yticklabels("")
        g.set_title(titulo)
        g.set_ylabel(variable_predict)
        g.set_xticklabels(['{:.0%}'.format(x) for x in vals])
        countv = 0
        for v in df.iloc[:,0]:
            g.text(np.mean([countv,countv+v]) - 0.03, 1 , '{:.1%}'.format(v), color='black', fontweight='bold')
            countv = countv + v
        g.legend(loc='upper center', bbox_to_anchor=(1.08, 1), shadow=True, ncol=1)

    def poder_predictivo_categorica(self, var:str):
        "Método para ver la distribución de una variable categórica respecto a la predecir"
        data = self.datos
        variable_predict = self.predecir
        df = pd.crosstab(index= data[var],columns=data[variable_predict])
        df = df.div(df.sum(axis=1),axis=0)
        titulo = "Distribución de la variable %s según la variable %s" % (var,variable_predict)
        g = df.plot(kind='barh',stacked=True,legend = True, figsize = (10,9), \
                    xlim = (0,1),title = titulo, width = 0.8)
        vals = g.get_xticks()
        g.set_xticklabels(['{:.0%}'.format(x) for x in vals])
        g.legend(loc='upper center', bbox_to_anchor=(1.08, 1), shadow=True, ncol=1)
        for bars in g.containers:
            plt.setp(bars, width=.9)
        for i in range(df.shape[0]):
            countv = 0
            for v in df.iloc[i]:
                g.text(np.mean([countv,countv+v]) - 0.03, i , '{:.1%}'.format(v), color='black', fontweight='bold')
                countv = countv + v


    def poder_predictivo_numerica(self,var:str):
        "Función para ver la distribución de una variable numérica respecto a la predecir"
        sns.FacetGrid(self.datos, hue=self.predecir, height=6).map(sns.kdeplot, var, shade=True).add_legend()

Como podemos ver en el siguiente grafico nos estamos enfrentando a un problema un poco desequilibrado, por que los resultados positivos son mucho mas frecuentes que los negativos, hablando en cifras un 30.6% mas

In [ ]:
analisisPredictivo = Analisis_Predictivo(datosTicTacToe, predecir=" Gana-x")
analisisPredictivo.distribucion_variable_predecir()

Aqui podemos ver el poder predictivo de las variables del dataset de tic tac toe:

In [ ]:
for x in datosTicTacToe:
    if x != " Gana-x":
        analisisPredictivo.poder_predictivo_numerica(x)

In [ ]:
bayes = GaussianNB()
analisisPredictivoBayes = Analisis_Predictivo(datosTicTacToe,predecir= " Gana-x",modelo=bayes,
                                       train_size= 0.8, random_state=0)
print("Las predicciones en Testing son: {}".format(analisisPredictivoBayes.fit_predict()))

In [ ]:
resultados = analisisPredictivoBayes.fit_predict_resultados()
precisionGlobal = pd.DataFrame({"Precisión Global" : [resultados['Precisión Global']]})
errorGlobal = pd.DataFrame({"Error Global" : [resultados['Error Global']]})

In [ ]:
precisonCategoria = resultados['Precisión por categoría']


Aqui abajo podemos ver que la precision global es de un 69% lo cual no esta mal, pero tenemos modelos que predicen mejor este problema.

El valor de la precision positiva tiene trampa por que basicamente predice casi todo positivo, por eso tiene un valor tan alto, y eso lo podemos confirmar cuando vemos el valor negativo que muy bajo 25%

In [ ]:
finalDataFrame = precisionGlobal.join(errorGlobal).join(precisonCategoria)
finalDataFrame

In [ ]:
prediccionADABosting = PrediccionADABoosting(datos=datosTicTacToe)
prediccionADABosting.entrenamiento(train_size=0.80)

In [ ]:
prediccionXGBoosting = PrediccionXGBoosting(datos=datosTicTacToe)
prediccionXGBoosting.entrenamiento(train_size=0.80)

In [ ]:
prediccionRandomForest = PrediccionRandomForest(datos=datosTicTacToe)
prediccionRandomForest.entrenamiento(train_size=0.80)

Aqui podemos ver que el resultado de los modelos como Decision Trees son mejoran mucho mas la prediccion

Teniendo una precision global de mas del 88% en contraste del 69% del metodo de bayes el mejor modelo es el de  Bosques Aleatorios

In [ ]:
reporteFinal = pd.concat([prediccionADABosting.reporte, prediccionXGBoosting.reporte, prediccionRandomForest.reporte])
reporteFinal

In [ ]:
datosWine = pd.read_csv("wine.csv")
datosWine["tipo"] = datosWine["tipo"].astype('category')

In [ ]:
bayes = GaussianNB()
analisisPredictivoBayesWine = Analisis_Predictivo(datosWine,predecir= "tipo",modelo=bayes,
                                       train_size= 0.75, random_state=0)

print("Las predicciones en Testing son: {}".format(analisisPredictivoBayesWine.fit_predict()))

En la proxima celda voy a mostrar el analisis del poder predictivo de cada variables para saber cuales son las 6 mas influyentes

In [ ]:
for x in datosWine:
    if x != "tipo":
        analisisPredictivoBayesWine.poder_predictivo_numerica(x)

In [ ]:
resultados = analisisPredictivoBayesWine.fit_predict_resultados()
precisionGlobal = pd.DataFrame({"Precisión Global" : [resultados['Precisión Global']]})
errorGlobal = pd.DataFrame({"Error Global" : [resultados['Error Global']]})
finalDataFrame = precisionGlobal.join(errorGlobal).join(precisonCategoria)
finalDataFrame


In [ ]:
prediccionADABosting = PrediccionADABoosting(datos=datosWine)
prediccionADABosting.entrenamiento(train_size=0.75)

In [ ]:
prediccionXGBoosting = PrediccionXGBoosting(datos=datosWine)
prediccionXGBoosting.entrenamiento(train_size=0.75)

In [ ]:
prediccionRandomForest = PrediccionRandomForest(datos=datosWine)
prediccionRandomForest.entrenamiento(train_size=0.75)

Aqui podemos ver la precision con otros metodos como los bosques aleatorios, al parecer tiene unos resultados bastantes similares, pero realmente hay que notar que los metodos basados en arboles de decision superan en gran medida al metodo de prediccion de bayes

In [ ]:
reporteFinal = pd.concat([prediccionADABosting.reporte, prediccionXGBoosting.reporte, prediccionRandomForest.reporte])
reporteFinal

Asi queda la prediccion con la variables mas importantes, realmente no hay mucha diferencia, pero parece que predice un poco peor pero muy poco

In [ ]:
bayes = GaussianNB()
analisisPredictivoBayesWineFiltrado = Analisis_Predictivo(datosWine,predecir= "tipo",predictoras= ["volatil.acidez", "cloruros", "total.sulfuro.dioxido",
                                             "libre.sulfuro.dioxido", "sulfitos", "densidad"], modelo=bayes,
                                       train_size= 0.75, random_state=0)

print("Las predicciones en Testing son: {}".format(analisisPredictivoBayesWineFiltrado.fit_predict()))
resultados = analisisPredictivoBayesWineFiltrado.fit_predict_resultados()
precisionGlobal = pd.DataFrame({"Precisión Global" : [resultados['Precisión Global']]})
errorGlobal = pd.DataFrame({"Error Global" : [resultados['Error Global']]})
finalDataFrame = precisionGlobal.join(errorGlobal).join(precisonCategoria)
finalDataFrame